# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

experiment_name = 'cifar-automl-experiment'
experiment = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Attach an AmlCompute cluster

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Use GPU instance because it is better suited for computer vision problems
amlcompute_cluster = "cifar-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                           vm_priority='lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster, compute_config)

compute_target.wait_for_completion(show_output=True)

## Dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

Reference: https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
from azureml.core import Dataset

# URL to the CIFAR-10 dataset
dataset_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

# Register the dataset using the Dataset class
dataset_name = "cifar10_dataset"
cifar10_dataset = Dataset.File.from_files(dataset_url)
cifar10_dataset.register(workspace=ws, name=dataset_name, create_new_version=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
from azureml.train.automl import AutoMLConfig

# TODO: Create AutoML config
automl_settings = {}
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15, # How much timeout?
    task='classification', # Classification, right?
    primary_metric='accuracy',
    training_data=cifar10_dataset,
    label_column_name='', # What is the label col
    n_cross_validations=5
    )

In [ ]:
# Submit the automl run to the experiment
automl_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve the best automl model
best_run, best_model = automl_run.get_output()
best_run_metrics = best_run.get_metrics()

print(f"Best run: {best_run}\n")
print(f"Best model: {best_model}\n")
print(f"Best run metrics: {best_run_metrics}\n")
print(f"Associated files: {best_run.get_file_names()}\n")

In [ ]:
# TODO: Download the model
# Downloading model from the run into the workspace
best_run.download_file('models/*.*')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model

# TODO: Change the `model_path`
model = Model.register(model_path="*.keras",
                       model_name="cirfar10-best-model",
                       tags={'topic': "computer_vision", 'type': "classification"},
                       description="CNN model on CIFAR-10 dataset to classify 32x32 images.",
                       workspace=ws)

### Deploy an Azure Container Instance Web Service

In [ ]:
service_name = 'cifar10-service'

# Use the `deployment_config` argument for custom config
service = Model.deploy(ws, service_name, [model], overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

### Test the Web Service Endpoint

In [ ]:
from utils import visualize
# TODO: Replace it with the consume script provided by AML studio
# Get input data (use the dumped json from the inference script) -> Show as img -> Inference
input_data = {
    "input_key": input_value
}

# Call the web service endpoint
result = service.run(input_data)
print("Web service response:", result)

# TODO: Get the predicted labels ID
predicted_labels = ...
# TODO: Get the test labels ID
test_labels = ...

visualize[result[0]]
print("Predicted labels: ", predicted_labels[0])
print("Ground truth: ", test_labels[0])

## Delete the Resource
TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
